### Functions for automating model selection through cross-validation.

In [1]:
import verde as vd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import pyproj
from shapely import geometry

from sources import source_code as f

%matplotlib widget

# Selecionando Area de Estudo
 - Este Jupyternotebook tem como base o tutorial do verde presente no respositorio do transform.

 - No Tutorial 4 definimos uma funçao que utiliza a funçao verde.get_region() para:
    - construir um dicionario de folhas de cartas litologicas e,
    - um dicionario de metadata (Informaçoes sobre os dados.)

In [2]:
dic_cartas,dic_raw_metadata = f.get_region('100k','SC23_Z','gama_line_1089',camada='litologia_100k')

importando dados geofisicos


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


# --- Iniciando seleção de área de estudo
24 folhas cartográfica selecionadas

Lista de chaves do dicionario de cartas: ['region', 'region_proj', 'raw_data', 'splines', 'scores', 'lito_splines', 'mean_score', 'litologia', 'cubic', 'lito_cubic']
Listando atributos dos dados geofisicos


100%|██████████| 12/12 [00:00<00:00, 289262.34it/s]

# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


Lista de chaves do dicionario de metadata bruto: ['Metadata', 'Lista_id', 'Lista_at_geof', 'Lista_at_geog', 'Lista_at_proj', 'Percentiles', 'Malha_cartografica']

# --- Início da iteração entre as folhas cartográficas #


0it [00:00, ?it/s]

SC23_Z_A_I geometry       POLYGON ((500000 8894587.508698922, 554798.412...
region                              (-45.0, -44.5, -10.5, -10.0)
region_proj    (500000.0, 554798.4121514614, 8839261.40022261...
Name: SC23_Z_A_I, dtype: object
# Folha de código: SC23_Z_A_I
 Atualizando dados brutos em dic_cartas['raw_data']
 com 50757 pontos de amostragem


## Dicionario de Metadados

In [ ]:
print(f'Chaves do dicionario de metadados: {dic_raw_metadata.keys()}')

Chaves do dicionario de metadados: dict_keys(['Metadata', 'Lista_id', 'Lista_at_geof', 'Lista_at_geog', 'Lista_at_proj', 'Percentiles', 'Malha_cartografica'])


### Visualizando Malha Cartografica

In [13]:
def labels(gdf):
    gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
    gdf['coords'] = [coords[0] for coords in gdf['coords']]

    for index, row in gdf.iterrows():
        plt.annotate(s=row['id_folha'], xy=row['coords'],horizontalalignment='center')

In [17]:
def plot_base(gdf,atributo=None,escala=None,nome=None):
    litologia = f.importar(escala,nome)

    if atributo:
        ax = litologia.plot('SIGLA')
        gdf.plot(atributo,ax=ax)
    else:
        ax = litologia.plot('SIGLA')
        gdf.plot(ax=ax)

In [16]:
area = dic_raw_metadata['Malha_cartografica']

In [15]:
f.plot_base(area.boundary,atributo=None,escala='litologia_100k',nome='Rio Paraim')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
labels(area)

## Dicionario dos dados litologicos e geofisicos

In [7]:
print(f'Chaves do dicionario principal: {list(dic_cartas.keys())}')

Chaves do dicionario principal: ['region', 'region_proj', 'raw_data', 'splines', 'scores', 'lito_splines', 'mean_score', 'litologia', 'cubic', 'lito_cubic']


### Definindo data

In [29]:
data = dic_cartas['raw_data']['SC23']

In [30]:
coordinates = (data.X.values, data.Y.values)

coordinates

(array([472115.74, 472115.9 , 472116.15, ..., 665135.62, 665140.29,
        665143.98]),
 array([8754877.13, 8754953.8 , 8755030.53, ..., 8782262.93, 8782185.57,
        8782108.02]))

In [31]:
dic_raw_metadata['Percentiles'].T

,count,mean,std,min,0.1%,10%,25%,50%,75%,99.5%,max
MDT,635173.0,508.893234,85.329898,290.6400,312.140000,400.9600,450.5400,501.3000,564.3900,722.290000,795.5500
KPERC,635173.0,0.520116,0.545391,-0.4100,-0.120000,0.0500,0.1400,0.3500,0.7200,2.990000,6.5200
eU,635173.0,1.107293,0.683094,-1.6400,-0.290000,0.3800,0.6500,1.0000,1.4400,3.880000,25.5000
eth,635173.0,9.470464,5.405466,-0.5600,0.600000,3.6200,5.7900,8.4000,11.9900,29.290000,70.1300
CTCOR,635173.0,836.493968,450.466561,-86.6500,72.351280,331.9800,513.8200,754.0300,1085.9200,2424.664200,5958.8200
THKRAZAO,635173.0,52.609745,92.576155,0.4532,1.701555,7.4646,12.3368,24.0520,51.7335,658.665466,1892.6940
UKRAZAO,635173.0,6.368725,10.713721,0.0118,0.045417,0.7330,1.3617,2.7836,6.1791,69.429660,217.3391
UTHRAZAO,635173.0,0.129238,0.070951,0.0023,0.005800,0.0583,0.0848,0.1182,0.1596,0.427100,1.8439


In [32]:
print(data.query('KPERC < 0')['KPERC'].count())
print(data.query('eU < 0')['eU'].count())

print(data.query('CTCOR < 0')['CTCOR'].count())
print(data.query('eth < 0')['eth'].count())

20280
5868
99
133


In [33]:
data.loc[data.KPERC < 0, 'KPERC'] = 0
data.loc[data.eU < 0, 'eU'] = 0
data.loc[data.eth < 0, 'eth'] = 0
data.loc[data.CTCOR < 0, 'CTCOR'] = 0

In [34]:
print(data.query('KPERC < 0')['KPERC'].count())
print(data.query('eU < 0')['eU'].count())

print(data.query('CTCOR < 0')['CTCOR'].count())
print(data.query('eth < 0')['eth'].count())

0
0
0
0


In [35]:
data_descrito = data.describe(percentiles= [0.01,0.1,0.25,0.5,0.75,0.995])

In [36]:
data_descrito.T

,count,mean,std,min,1%,10%,25%,50%,75%,99.5%,max
X,635173.0,5.759517e+05,56426.671892,4.720585e+05,4.741124e+05,4.936205e+05,5.261256e+05,5.806173e+05,6.256214e+05,6.646191e+05,6.651544e+05
Y,635173.0,8.840769e+06,42677.134216,8.754866e+06,8.759358e+06,8.787244e+06,8.806403e+06,8.838350e+06,8.871277e+06,8.935057e+06,8.937363e+06
MDT,635173.0,5.088932e+02,85.329898,2.906400e+02,3.359900e+02,4.009600e+02,4.505400e+02,5.013000e+02,5.643900e+02,7.222900e+02,7.955500e+02
KPERC,635173.0,5.213949e-01,0.544087,0.000000e+00,0.000000e+00,5.000000e-02,1.400000e-01,3.500000e-01,7.200000e-01,2.990000e+00,6.520000e+00
eU,635173.0,1.108532e+00,0.680827,0.000000e+00,1.000000e-02,3.800000e-01,6.500000e-01,1.000000e+00,1.440000e+00,3.880000e+00,2.550000e+01
eth,635173.0,9.470497e+00,5.405406,0.000000e+00,1.610000e+00,3.620000e+00,5.790000e+00,8.400000e+00,1.199000e+01,2.929000e+01,7.013000e+01
CTCOR,635173.0,8.364969e+02,450.460998,0.000000e+00,1.487200e+02,3.319800e+02,5.138200e+02,7.540300e+02,1.085920e+03,2.424664e+03,5.958820e+03
THKRAZAO,635173.0,5.260974e+01,92.576155,4.532000e-01,3.246000e+00,7.464600e+00,1.233680e+01,2.405200e+01,5.173350e+01,6.586655e+02,1.892694e+03
UKRAZAO,635173.0,6.368725e+00,10.713721,1.180000e-02,1.816000e-01,7.330000e-01,1.361700e+00,2.783600e+00,6.179100e+00,6.942966e+01,2.173391e+02
UTHRAZAO,635173.0,1.292379e-01,0.070951,2.300000e-03,1.460000e-02,5.830000e-02,8.480000e-02,1.182000e-01,1.596000e-01,4.271000e-01,1.843900e+00


In [37]:
print(f"Lista de atributos geofisicos: {dic_raw_metadata['Lista_at_geof']}")

Lista de atributos geofisicos: ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']


In [38]:
# LISTA DE TITULOS 
titles =   ['MDT (m)',
            'K (%)',
            'U (ppm)',
            'Th (ppm)',
            'Contagem Total (μR/h)',
            'Th (ppm) / K (%)',
            'U (ppm) / K (%)',
            'U (ppm) / Th (ppm)']

# DICIONARIO DE TITULOS
dic_titles = {}
for f, t in zip(dic_raw_metadata['Lista_at_geof'], titles):
    dic_titles[f] = t

dic_titles

{'MDT': 'MDT (m)',
 'KPERC': 'K (%)',
 'eU': 'U (ppm)',
 'eth': 'Th (ppm)',
 'CTCOR': 'Contagem Total (μR/h)',
 'THKRAZAO': 'Th (ppm) / K (%)',
 'UKRAZAO': 'U (ppm) / K (%)',
 'UTHRAZAO': 'U (ppm) / Th (ppm)'}

## Plotando dados Brutos

In [40]:
def raw_plot(data):
    fig, axs = plt.subplots(nrows = 4, ncols = 2, figsize = (12, 12))

    X,Y = data.X,data.Y

    for ax, f in zip(axs.flat, dic_titles):
        vmin=data_descrito[f]['1%']
        vmax=data_descrito[f]['99.5%']

        g = ax.scatter(c=data[f], x = X,
                                y = Y,
                                cmap = 'rainbow',
                                s    = 0.5,
                                vmax =vmax, vmin=vmin)

        fig.colorbar(g, ax = ax)
        ax.set_title(str(dic_titles[f]), size =9)
        ax.axis('scaled')

    #fig.delaxes(axs[2][1])

    #plt.tight_layout()

In [41]:
raw_plot(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# Block Reduction
reducer_mean_490 = vd.BlockReduce(np.mean, spacing= 490)
reducer_mean_499 = vd.BlockReduce(np.mean, spacing= 499)
reducer_mean_500 = vd.BlockReduce(np.mean, spacing= 500)

In [43]:
# Block Reduction
reducer_median_490 = vd.BlockReduce(np.median, spacing= 490)
reducer_median_499 = vd.BlockReduce(np.median, spacing= 499)
reducer_median_500 = vd.BlockReduce(np.median, spacing= 500)

In [44]:
lista_blocks = [reducer_mean_490,reducer_mean_499,reducer_mean_500,
                reducer_median_490,reducer_median_499,reducer_median_500]

## Plotando dados reduzidos em medias moveis de 500, 499 e 490 metros

In [46]:
fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize = (16, 16),sharex='all',sharey='all')

for ax, f in zip(axs.flat, lista_blocks):
    b_coords, b_geof = f.filter(coordinates, data.MDT)

    g = ax.scatter(c    = b_geof, x = b_coords[0],
                                y = b_coords[1],
                             cmap = 'rainbow',
                             s    = 5)
    fig.colorbar(g,ax=ax)
    ax.set_title(f,size=4)
    ax.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
reducer_mean_490 = vd.BlockReduce(np.mean, spacing= 1000)
b_coords, b_geof = reducer_mean_490.filter(coordinates, data.UTHRAZAO)

In [70]:
spline = vd.Spline()

In [71]:
spline.fit(b_coords, b_geof)

In [51]:
predicted = spline.predict(coordinates)

In [62]:
dic_raw_metadata['Percentiles']['UTHRAZAO']

count    635173.000000
mean          0.129238
std           0.070951
min           0.002300
0.1%          0.005800
10%           0.058300
25%           0.084800
50%           0.118200
75%           0.159600
99.5%         0.427100
max           1.843900
Name: UTHRAZAO, dtype: float64

In [64]:
plt.figure(figsize=(8,8))
descrito =  dic_raw_metadata['Percentiles']['UTHRAZAO']
plt.scatter(coordinates[0],coordinates[1],
            c=predicted,
            cmap='rainbow',
            s=1,
            vmin = descrito['0.1%'],
            vmax=descrito['99.5%'])
plt.axis('scaled')
plt.title('Valores preditos para os mesmos pontos amostrados')
plt.colorbar(label='UTHRAZAO')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
dic_cartas.keys()

dict_keys(['region', 'region_proj', 'raw_data', 'splines', 'scores', 'lito_splines', 'mean_score', 'litologia', 'cubic', 'lito_cubic'])

In [ ]:
a = sqrt((2 - 4)**2)

In [58]:
residuals = sqrt((data.UTHRAZAO - predicted)**2)

In [59]:
scale = vd.maxabs(residuals)

plt.figure(figsize=(7,7))

plt.scatter(coordinates[0],coordinates[1],
            c=residuals,
            cmap='RdBu_r',
            s=2,
            vmin=-scale,vmax=scale)
plt.axis('scaled')
plt.title('Residuals: Dado amostrado - Dado predito')
plt.colorbar(label='UTHRAZAO')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 250,pixel_register=True)

In [67]:
grid_UTHRAZAO = spline.predict(grid_coords)

In [68]:
plt.figure(figsize=(8,8))
plt.scatter(grid_coords[0], grid_coords[1],
            c=grid_UTHRAZAO,
            cmap='rainbow',
            s=2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
grid = spline.grid(spacing=250, data_names=['UTHRAZAO'])
grid

<xarray.Dataset>
Dimensions:   (northing: 221, easting: 219)
Coordinates:
  * easting   (easting) float64 5.001e+05 5.003e+05 ... 5.544e+05 5.547e+05
  * northing  (northing) float64 8.784e+06 8.784e+06 ... 8.839e+06 8.839e+06
Data variables:
    UTHRAZAO  (northing, easting) float64 0.1177 0.1284 0.1457 ... 0.0906 0.1254
Attributes:
    metadata:  Generated by Spline()

In [ ]:
plt.figure()
grid.UTHRAZAO.plot(cmap='rainbow')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499931.6324672345, 554778.1632470512, 8784061.033629868, 8839209.242560606)

In [ ]:
grid = vd.distance_mask(coordinates, maxdist=500, grid=grid)
grid

grid.UTHRAZAO.plot(figsize=(7,7),cmap='rainbow')
plt.axis('scaled')

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/xarray/plot/utils.py:429: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  _, ax = plt.subplots(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499931.6324672345, 554778.1632470512, 8784061.033629868, 8839209.242560606)

In [65]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.mean, spacing=1000)),
    ('spline', vd.Spline())
])

In [ ]:
chain.fit(coordinates, data.UTHRAZAO)

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(reduction=<function mean at 0x7f8ff81cb3a0>,
                          spacing=1000)),
             ('spline', Spline())])

In [ ]:
grid = chain.grid(spacing=250, data_names=['UTHRAZAO'])
grid_mask = vd.distance_mask(coordinates, maxdist=500, grid=grid)
grid_mask

<xarray.Dataset>
Dimensions:   (northing: 222, easting: 220)
Coordinates:
  * easting   (easting) float64 5e+05 5.003e+05 ... 5.544e+05 5.547e+05
  * northing  (northing) float64 8.784e+06 8.784e+06 ... 8.839e+06 8.839e+06
Data variables:
    UTHRAZAO  (northing, easting) float64 0.1301 0.1312 0.133 ... 0.1108 0.1121
Attributes:
    metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...

In [ ]:
df_THKRAZAO=grid_mask.to_dataframe()
df_THKRAZAO_descrito = df_THKRAZAO.describe(percentiles=(0.01,0.1,0.25,0.5,0.75,0.995))

In [ ]:
df_THKRAZAO_descrito

,UTHRAZAO
count,48840.000000
mean,0.120143
std,0.038541
min,0.029307
1%,0.064182
10%,0.077894
25%,0.091941
50%,0.115054
75%,0.139801
99.5%,0.267650


In [ ]:
grid_mask.UTHRAZAO.plot(figsize=(8,8), cmap='rainbow',vmin=df_THKRAZAO_descrito.UTHRAZAO.min(),
                                                      vmax=df_THKRAZAO_descrito.UTHRAZAO.max())
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499880.4371004566, 554820.1128995434, 8783850.761764705, 8839429.898235295)

### Model Validation

### Escondendo 10% dos dados antes de inserir no chain.fit()

In [ ]:
train, test = vd.train_test_split(coordinates, data.UTHRAZAO,
                                 test_size = 0.1)

plt.figure(figsize=(12,12))
plt.title(label='Dados Escondidos(Vermelho) Dados inseridos no modelo de Splines(Azul)')
plt.plot(train[0][0], train[0][1], '.b', markersize=4)
plt.plot(test[0][0], test[0][1], '.r', markersize=4)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497270.8025, 557429.7475, 8781209.501, 8842071.159)

In [ ]:
chain.fit(*train)
# *train   ---> chain.fit(train[0], train[1])

chain.score(*test)


0.6673176584341012

In [ ]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=4)),
    ('reduce', vd.BlockReduce(np.mean, spacing=1500)),
    ('spline', vd.Spline())
])

In [ ]:
train, test = vd.train_test_split(coordinates, data.UTHRAZAO, test_size=0.1, spacing=500)

plt.figure(figsize=(9,9))
plt.plot(train[0][0], train[0][1], '.b', markersize=10)
plt.plot(test[0][0], test[0][1], '.r', markersize=10)
plt.axis('scaled')

chain.fit(*train)

chain.score(*test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.339774914835025

# Script

In [ ]:
from sources import source_code as f

In [ ]:
dic_cartas['region_proj'][i]

(500000.0, 527356.0359910944, 8811651.835175423, 8839304.90587394)

In [ ]:
dic_cartas,dic_raw_metadata = f.get_region('100k','SC23_Z_A_IV','gama_1089')

lista_id = dic_raw_metadata['Lista_id']
print(list(lista_id))

for i in lista_id:
    print(f"folha {i}")
    data = dic_cartas['raw_data'][i]

    coordinates = (data.X.values, data.Y.values)
    region = dic_cartas['region_proj'][i]

    chain = vd.Chain([('trend',  vd.Trend(degree=1)),
                     ('reduce', vd.BlockReduce(np.mean, spacing=1000)),
                     ('spline', vd.Spline())
                    ])

    cv     = vd.BlockKFold(spacing=500,
                            n_splits=10,
                            shuffle=True)

    scores = vd.cross_val_score(chain,
                            coordinates,
                            data.eU,
                            cv=cv,
                            delayed=True)

    import dask
    mean_score = dask.delayed(np.mean)(scores)
    print("Delayed mean:", mean_score)

    mean_score = mean_score.compute()

    print(mean_score)

    print("")

importando dados geofisicos
# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos
['SC23_Z_A_IV']
folha SC23_Z_A_IV
Delayed mean: Delayed('mean-cdb89650-64c9-4f20-aa81-11a8e104faff')
0.511437021127595



In [ ]:
dic_cartas,dic_raw_metadata = f.get_region('100k','SC23_Z_A_IV','gama_1089')

lista_id = dic_raw_metadata['Lista_id']
print(list(lista_id))

for i in lista_id:
    print(f"folha {i}")
    data = dic_cartas['raw_data'][i]

    coordinates = (data.X.values, data.Y.values)
    region = dic_cartas['region_proj'][i]

    chain = vd.Chain([('trend',  vd.Trend(degree=3)),
                     ('reduce', vd.BlockReduce(np.mean, spacing=2000)),
                     ('spline', vd.Spline())
                    ])

    cv     = vd.BlockKFold(spacing=100,
                            n_splits=50,
                            shuffle=True)

    scores = vd.cross_val_score(chain,
                            coordinates,
                            data.eU,
                            cv=cv,
                            delayed=True)

    import dask
    mean_score = dask.delayed(np.mean)(scores)
    print("Delayed mean:", mean_score)

    mean_score = mean_score.compute()

    print(mean_score)

    print("")

importando dados geofisicos
# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos
['SC23_Z_A_IV']
folha SC23_Z_A_IV
Delayed mean: Delayed('mean-d5ad70cd-a5ef-46f1-83e5-bc018883158a')
0.4794616449012649



In [ ]:
dic_cartas.keys()

dict_keys(['region', 'region_proj', 'raw_data', 'interpolado', 'scores', 'lito_geof', 'mean_score'])

In [ ]:
dic_cartas['interpolado']['SC23_Z_A_IV']

''

In [ ]:
lista_id = dic_raw_metadata['Lista_id']
print(list(lista_id))
lista_geof = dic_raw_metadata['Lista_at_geof']
print(list(lista_geof))

['SC23_Z_A_IV']
['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']


In [ ]:
for index in lista_id:
    print(f"folha {index}")
    data = dic_cartas['raw_data'][index]

    coordinates = (data.X.values, data.Y.values)
    region = dic_cartas['region_proj'][index]

    chain = vd.Chain([('trend',  vd.Trend(degree=1)),
                     ('reduce', vd.BlockReduce(np.mean, spacing=2000)),
                     ('spline', vd.Spline())
                    ])


    print(f"# Folha de código: {index}")
    print(f" Atualizando dados brutos em dic_cartas['raw_data']")
    x = {index:data}
    data_list[0]['raw_data'].update(x) 
    print(f" com {len(data)} pontos de contagens radiométricas coletados com linhas de voo de {spacing} metros")
    
    # ADICIONANDO ATRIBUTOS GEOFÍSICOS AO DICIONÁRIO INTERPOLADO
    interpolado={}
    for atributo in lista_geof:
        x = {atributo:''}
        interpolado.update(x)
    #print(f" Construindo dic_cartas['interpolado'] vazio com os atributos geofísicos")
        
    # ADICIONANDO ATRIBUTOS AO DICIONÁRIO SCORES
    scores={}
    for atributo in lista_geof:
        y = {atributo:''}
        scores.update(y)

    mean_score={}
    for atributo in lista_geof:
        x = {atributo:''}
        mean_score.update(x)    

    #print(f" Construindo dicionário vazio de score do cross validation")
    
    for i in lista_geof:
        cv     = vd.BlockKFold(spacing=500,
                                n_splits=10,
                                shuffle=True)

        scores[i] = vd.cross_val_score(chain,
                                coordinates,
                                data[i],
                                cv=cv,
                                delayed=True)

        import dask
        
        mean_score[i] = dask.delayed(np.mean)(scores[i])
        print("Delayed mean:", mean_score)

        mean_score[i] = mean_score[i].compute()
        print(mean_score)


        y = {index:scores}
        dic_cartas['scores'].update(y)
        x = {index:mean_score}
        dic_cartas['mean_score'].update(x)
        print(f"# Folha {index} atualizada ao dicionário")

        print("")

folha SC23_Z_A_IV
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']


NameError: name 'data_list' is not defined

In [ ]:
grid_coords = vd.grid_coordinates(region, spacing = 250,pixel_register=True)
chain.fit(grid_coords)